## **Задача 10 - SOAP**
### Общее описание задачи - Продвинутый уровень
Требуется реализовать программу, которая будет получать данные при помощи методов [любого SOAP сервиса в данном списке](https://www.postman.com/cs-demo/public-soap-apis/collection/eebj1yq/public-soap-apis). Обратите внимание, что для каждого сервиса есть несколько POST методов. То есть ваша программа должна полностью утилизировать весь функционал выбранного вами SOAP сервиса


In [1]:
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from ipywidgets import interact, IntSlider, FloatSlider, Layout, HTML

In [2]:
def get_payload(number, service="n2w"):
    if service == "n2w":
        tag_name = "NumberToWords"
        num_tag = "ubiNum"
    elif service == "n2d":
        tag_name = "NumberToDollars"
        num_tag = "dNum"
    else:
        raise ValueError("Invalid service. Choose 'n2w' for Number to Words or 'n2d' for Number to Dollars.")
    
    return f"""<?xml version="1.0" encoding="utf-8"?>
               <soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
                 <soap:Body>
                   <{tag_name} xmlns="http://www.dataaccess.com/webservicesserver/">
                     <{num_tag}>{number}</{num_tag}>
                   </{tag_name}>
                 </soap:Body>
               </soap:Envelope>"""

In [3]:
def parse_response(response, service="n2w"):
    root = ET.fromstring(response.content)
    namespace = {'m': 'http://www.dataaccess.com/webservicesserver/'}
    
    if service == "n2w":
        result_tag = "NumberToWordsResult"
    elif service == "n2d":
        result_tag = "NumberToDollarsResult"
    else:
        raise ValueError("Invalid service. Choose 'n2w' for Number to Words or 'n2d' for Number to Dollars.")
    
    result = root.find(f".//m:{result_tag}", namespace).text
    return result

In [4]:
def number_to_words(number):
    url = "https://www.dataaccess.com/webservicesserver/NumberConversion.wso"
    payload = get_payload(number, 'n2w')
    headers = {
        'Content-Type': 'text/xml; charset=utf-8'
    }
    
    response = requests.post(url, headers=headers, data=payload)
    response.raise_for_status()
    return parse_response(response, 'n2w')

In [5]:
def number_to_dollars(number):
    url = "https://www.dataaccess.com/webservicesserver/NumberConversion.wso"
    payload = get_payload(number, 'n2d')
    headers = {
        'Content-Type': 'text/xml; charset=utf-8'
    }
    
    response = requests.post(url, headers=headers, data=payload)
    response.raise_for_status()
    return parse_response(response, 'n2d')

In [6]:
# def display_services(n2w, n2d):
#     n2w_conv = number_to_words(n2w)
#     n2d_conv = number_to_dollars(n2d)
#     print(f'{n2w} -> {n2w_conv}', f'{n2d} -> {n2d_conv}', sep='\n')

In [7]:
def display_services(n2w, n2d):
    global result
    n2w_conv = number_to_words(n2w)
    n2d_conv = number_to_dollars(n2d)
    result_html = f"""
    <link href="https://fonts.googleapis.com/css2?family=Fira+Mono:wght@400;700&display=swap" rel="stylesheet">
    <div style="font-family: 'Fira Code', monospace;">
        {n2w} -> <strong>{n2w_conv}</strong><br>
        {n2d} -> <strong>{n2d_conv}</strong>
    </div>
    """
    result_widget.value = result_html

In [8]:
n2w = IntSlider(
    min=0, 
    max=1_000_000_000_000, 
    step=1, 
    continuous_update=False, 
    layout=Layout(width='95%'), 
    description='N2Words',
    tooltip='Number To Words',
    value=922813371488
)

n2d = FloatSlider(
    min=0.01, 
    max=1_000_000_000_000, 
    step=0.01, 
    continuous_update=False, 
    layout=Layout(width='95%'), 
    description='N2Dollars',
    tooltip='Number To Dollars (And Cents)',
    value=92281337.1488
)

result_widget = HTML()

_ = interact(display_services, n2w=n2w, n2d=n2d)
display(result_widget)

interactive(children=(IntSlider(value=922813371488, continuous_update=False, description='N2Words', layout=Lay…

HTML(value='\n    <link href="https://fonts.googleapis.com/css2?family=Fira+Mono:wght@400;700&display=swap" re…